# Classification of the Fire Using OPLS-DA

#### In this notebook, we use statistical learning for modeling the evolution of the wildfires and predict when are going to become critical.
#### To achieve this, we use the Orthogonal Partial Least Square Discriminant Analysis (OPLS-DA) technique.

#### We consider a binary classification problem with the following classes:

#### - Class -1: Non-critical fire (burns less than 10 000 acres)
#### - Class 1: Critical fire (burns more than 10 000 acres)

#### Also, we consider the scenario where we want to predict if the fire is going to grow critically within the next 6 hours.

#### This horizon of prediction of 6 hours is arbitrary (you could change it), and corresponds to the delay that could be to mobilise the appropriate resources to deal with the fire before it becomes critical.

### Install some required packages

In [ ]:
!pip install pyopls

### Import some packages

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict, LeaveOneOut
from sklearn.metrics import r2_score, accuracy_score, classification_report

from pyopls import OPLS
from pyopls import OPLSValidator, OPLSDAValidator

import ast
import time
from collections import Counter

### Global variable

In [ ]:
RESULTS = '../results'

# STEP 1: Prediction at 6 hours horizon

## STEP 1.0: Load the data

In [ ]:
%%time
train_df_6h = pd.read_csv(os.path.join(RESULTS, 'extracted-features-ComprehensiveFCParameters-filtered-target-horizon-6h-train.csv'), index_col=0)
train_df_6h.shape

In [ ]:
train_df_6h.head(2)

In [ ]:
%%time
val_df_6h = pd.read_csv(os.path.join(RESULTS, 'extracted-features-ComprehensiveFCParameters-filtered-target-horizon-6h-val.csv'), index_col=0)
val_df_6h.shape

In [ ]:
val_df_6h.head(2)

In [ ]:
%%time
test_df_6h = pd.read_csv(os.path.join(RESULTS, 'extracted-features-ComprehensiveFCParameters-filtered-target-horizon-6h-test.csv'), index_col=0)
test_df_6h.shape

In [ ]:
test_df_6h.head(2)

## STEP 1.1: Modeling

### Project orthogonally the data into the latent space (O-PLS)

In [ ]:
X_cols = [c for c in train_df_6h.columns if c!='target']
y_col = 'target'

X_train_6h = train_df_6h[X_cols]
y_train_6h = train_df_6h[y_col]

# because we will use cross-validation here,
# we concatenate the validation and test datasets
# we consider the resulting dataset to test the model
X_test_6h = pd.concat([test_df_6h, val_df_6h])
X_test_6h = X_test_6h.sample(frac=1)
y_test_6h = X_test_6h[y_col]
X_test_6h = X_test_6h[X_cols]

In [ ]:
X_train_6h.shape

### Distribution of the classes

#### Train

In [ ]:
y_train_6h.value_counts(normalize=True)

#### Validation

#### Test

In [ ]:
y_test_6h.value_counts(normalize=True)

## O-PLS Model

#### Running this cell could take a few minutes...

In [ ]:
%%time
ncomp = 50   # using cross-validation, we determined this value as the optimum. But, this could be changed.
opls_6h = OPLS(ncomp)
Z_train_6h = opls_6h.fit_transform(X_train_6h, y_train_6h)

print(Z_train_6h.shape)

#### Check the goodness of the transformation
#### Lower values mean that most variance is removed

#### Running this cell could take a few minutes...

In [ ]:
%%time
opls_6h.score(X_train_6h)

In [ ]:
%%time
pls = PLSRegression(1)

y_pred = cross_val_predict(pls, X_train_6h, y_train_6h, cv=100, n_jobs=-1)
q_squared = r2_score(y_train_6h, y_pred)
dq_squared = r2_score(y_train_6h, np.clip(y_pred, -1, 1))
accuracy = accuracy_score(y_train_6h, np.sign(y_pred))

processed_y_pred = cross_val_predict(pls, Z_train_6h, y_train_6h, cv=100, n_jobs=-1)
processed_q_squared = r2_score(y_train_6h, processed_y_pred)  # 0.981
processed_dq_squared = r2_score(y_train_6h, np.clip(processed_y_pred, -1, 1))
processed_accuracy = accuracy_score(y_train_6h, np.sign(processed_y_pred))

fpr, tpr, thresholds = roc_curve(y_train_6h, y_pred)
roc_auc = roc_auc_score(y_train_6h, y_pred)
proc_fpr, proc_tpr, proc_thresholds = roc_curve(y_train_6h, processed_y_pred)
proc_roc_auc = roc_auc_score(y_train_6h, processed_y_pred)

plt.figure(figsize=(20, 10))
plt.plot(fpr, tpr, lw=2, color='blue', label=f'Unprocessed (AUC={roc_auc:.4f})')
plt.plot(proc_fpr, proc_tpr, lw=2, color='red',
         label=f'{ncomp}-component OPLS (AUC={proc_roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

### Fit the model

In [ ]:
%%time
pls = PLSRegression(5)
pls.fit(Z_train_6h, y_train_6h)

### Plot the transformed data into the latent space

In [ ]:
%%time
plt.figure(figsize=(20, 10))
df = pd.DataFrame(np.column_stack([pls.x_scores_, opls_6h.T_ortho_[:, 0]]),
                  index=X_train_6h.index, columns=['t', 't_ortho'])                           
pos_df = df[y_train_6h==1]
neg_df = df[y_train_6h==-1]
plt.scatter(neg_df['t'], neg_df['t_ortho'], c='blue', label='Non-critical fire')
plt.scatter(pos_df['t'], pos_df['t_ortho'], c='red', label='Critical fire')
plt.title('PLS Scores')
plt.xlabel('t_ortho')
plt.ylabel('t')
plt.legend(loc='upper right')
plt.show()

### Make predictions on the test dataset with the fitted model

In [ ]:
%%time
Z_test_6h = opls_6h.transform(X_test_6h)
y_pred_test_6h = pls.predict(Z_test_6h)
fpr_test_6h, tpr_test_6h, thresholds_test_6h = roc_curve(y_test_6h, y_pred_test_6h)
roc_auc_test_6h = roc_auc_score(y_test_6h, y_pred_test_6h)

plt.figure(figsize=(20, 10))
plt.plot(fpr_test_6h, tpr_test_6h, lw=2, color='red', label=f'Test (AUC={roc_auc_test_6h:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

## Conclusion

### We can see that the AUC of 0.66 is relatively low.
### This means that there is little information in the data. The pedictors are not correlated enough to the target. 
### This results in a low predictive power of the model.
### Finally, we could improve the model by collecting ground truth historical data on the evolution of fires, for example, the number of acres burnt each hour.